In [15]:
from data import read_squad_examples,convert_examples_to_features
from tokenization import BertTokenizer
import modeling
import torch

tokenizer = BertTokenizer('bert_pretrain\\vocab.txt', do_lower_case=True, max_len=384) # for bert large
config = modeling.BertConfig.from_json_file('bert_pretrain\\bert_config.json')
# Padding for divisibility by 8
# if config.vocab_size % 8 != 0:
#     config.vocab_size += 8 - (config.vocab_size % 8)

device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")
model = modeling.BertForQuestionAnswering(config)
checkpoint = torch.load('out_dir\pytorch_model.bin', map_location='cpu')
checkpoint = checkpoint["model"] if "model" in checkpoint.keys() else checkpoint
model.load_state_dict(checkpoint, strict=False)
model.to(device)


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [12]:
from tqdm import tqdm
import collections
from torch.utils.data import TensorDataset,SequentialSampler,DataLoader

RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

eval_examples = read_squad_examples(
    input_file='squad\one\one.json', is_training=False, version_2_with_negative=False)
eval_features = convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=384,
    doc_stride=126,
    max_query_length=60,
    is_training=False)

all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_example_index)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=2)


In [13]:
all_results = []
device = 'cuda'
for input_ids, input_mask, segment_ids, example_indices in tqdm(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    with torch.no_grad():
        batch_start_logits, batch_end_logits = model(input_ids, segment_ids, input_mask)
    for i, example_index in enumerate(example_indices):
        start_logits = batch_start_logits[i].detach().cpu().tolist()
        end_logits = batch_end_logits[i].detach().cpu().tolist()
        eval_feature = eval_features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        all_results.append(RawResult(unique_id=unique_id,
                                    start_logits=start_logits,
                                    end_logits=end_logits))

Evaluating: 100%|██████████| 467/467 [00:05<00:00, 86.60it/s]


In [14]:
import os
from run_squad import get_answers
import json
class args:
    do_lower_case = False
    verbose_logging = False
    max_answer_length = 30
    n_best_size = 20
    version_2_with_negative = False
    
output_prediction_file = os.path.join('squad\one', "predictions.json")
output_nbest_file = os.path.join('squad\one', "nbest_predictions.json")

answers, nbest_answers = get_answers(eval_examples, eval_features, all_results, args)
with open(output_prediction_file, "w") as f:
    f.write(json.dumps(answers, indent=4) + "\n")
with open(output_nbest_file, "w") as f:
    f.write(json.dumps(nbest_answers, indent=4) + "\n")